In [106]:
# Name: Jiajin Liang
# Email: jil904@ucsd.edu
import numpy as np
x = np.load("assignment7_X.npy")
w1 = np.load("assignment7_W1.npy")
w2 = np.load("assignment7_W2.npy")
w1 = w1.astype(float)
w2 = w2.astype(float)
x = x.astype(float)
print(x.shape,w1.shape,w2.shape)

class Node:
    def __init__(self,f,input_list):
        self.value = f(input_list)
        self.df_values = []
        self.edge_dfvalues = []
    def backprop(self,df_list,backp_input_list,prev_dfvalue_list):
        for df in df_list:
            self.df_values.append(df(backp_input_list))
        #print(self.df_values,df_list)
        self.dfvalue = np.array(self.df_values).dot(np.array(prev_dfvalue_list))
        for i in range(len(self.df_values)):
            self.edge_dfvalues.append(self.df_values[i] * prev_dfvalue_list[i])
    def tensor_chain_backprop(self,df,bpinput,prevdf):
        self.dfvalue = np.zeros((15,15,15))
        tensor = df(bpinput)
        for i in range(15):
            for j in range(15):
                self.dfvalue[i] += tensor[j]*prevdf[i][j]
    def simple_backprop(self,df,bpinput):
        self.dfvalue = df(bpinput)
    def printnode(self):
        print("forward value: ", self.value, "\ndfvalue (the sum of derivatives of all outgoing edges/the partial derivative of this node with respect to f): ", self.dfvalue)
        for i in range(len(self.edge_dfvalues)):
            print("Derivative of outgoing Edge #", i, " of this node: ", self.edge_dfvalues[i])
class inputNode(Node):
    def __init__(self,inputv):
        self.value = inputv
        self.df_values = []
        self.edge_dfvalues = []
        
def fh1(input_list):
    return input_list[1].dot(input_list[0])
def fh2(input_list):
    return input_list[0].dot(input_list[1])
def dh2dh1(bpinput):
    return bpinput
def dh2dw2(bpint):
    toRet = np.zeros((15,15,15))
    for i in range(15):
        toRet[i][i] = bpinput.T
    return toRet
def dh1dw1(bpinput):
    toRet = np.zeros((15,15,15))
    for i in range(15):
        toRet[i][i] = bpinput.T
    return toRet
        
nodexi = inputNode(x)
nodexi.value
nodew1 = inputNode(w1)
nodew2 = inputNode(w2)
#nodeb1 = 
nodeh1 = Node(fh1,[nodexi.value,nodew1.value])
nodeh2 = Node(fh2,[nodew2.value,nodeh1.value])

#nodeh2.tensor_chain_backprop([dh2dh1,dh2dw2],[nodeh1.value,nodew2.value],[np.ones((15,15))])
nodeh1.simple_backprop(dh2dh1,nodew2.value)
nodew2.simple_backprop(dh2dw2,nodeh1.value)
nodew1.tensor_chain_backprop(dh1dw1,nodexi.value,nodew2.value)

# compute numerical partial derivative tensor to compare
h = 1e-3
f = w2.dot((w1.dot(x)))
w1_partial_deriv_tensor = np.ones((15,15,15))
for i in range (15):
    for j in range(15):
        w1 = np.load("assignment7_W1.npy")
        w1 = w1.astype(float)
        w1[i][j] += h
        partial_deriv = (w2.dot((w1.dot(x))) - f)/h
        #print(w1)
        for k in range(15):
            w1_partial_deriv_tensor[k][i][j] = partial_deriv[k]
w1 = np.load("assignment7_W1.npy")
h1 = w1.dot(x)
w2_partial_deriv_tensor = np.ones((15,15,15))
for i in range (15):
    for j in range(15):
        w2 = np.load("assignment7_W2.npy")
        w2 = w2.astype(float)
        w2[i][j] += h
        partial_deriv = (w2.dot(h1) - f)/h
        #print(w2)
        for k in range(15):
            w2_partial_deriv_tensor[k][i][j] = partial_deriv[k]
print("dfdw1:")
print(nodew1.dfvalue)
print("numerical dfdw1:")
print(w1_partial_deriv_tensor)
print("dfdw2:")
print(nodew2.dfvalue)
print("numerical dfdw2:")
print(w2_partial_deriv_tensor)
#print("dh2dh1:")
#print(nodeh1.dfvalue)
#print(w2)

(15,) (15, 15) (15, 15)
dfdw1:
[[[  0.    0.    0.  ...   0.    0.    0. ]
  [  0.5   0.5   0.5 ...   0.5   0.5   0.5]
  [  1.    1.    1.  ...   1.    1.    1. ]
  ...
  [  6.    6.    6.  ...   6.    6.    6. ]
  [  6.5   6.5   6.5 ...   6.5   6.5   6.5]
  [  7.    7.    7.  ...   7.    7.    7. ]]

 [[  7.5   7.5   7.5 ...   7.5   7.5   7.5]
  [  8.    8.    8.  ...   8.    8.    8. ]
  [  8.5   8.5   8.5 ...   8.5   8.5   8.5]
  ...
  [ 13.5  13.5  13.5 ...  13.5  13.5  13.5]
  [ 14.   14.   14.  ...  14.   14.   14. ]
  [ 14.5  14.5  14.5 ...  14.5  14.5  14.5]]

 [[ 15.   15.   15.  ...  15.   15.   15. ]
  [ 15.5  15.5  15.5 ...  15.5  15.5  15.5]
  [ 16.   16.   16.  ...  16.   16.   16. ]
  ...
  [ 21.   21.   21.  ...  21.   21.   21. ]
  [ 21.5  21.5  21.5 ...  21.5  21.5  21.5]
  [ 22.   22.   22.  ...  22.   22.   22. ]]

 ...

 [[ 90.   90.   90.  ...  90.   90.   90. ]
  [ 90.5  90.5  90.5 ...  90.5  90.5  90.5]
  [ 91.   91.   91.  ...  91.   91.   91. ]
  ...
  [ 96.  